In [1]:
import re
import json
    
class StraceOutput:
    regular_expressions=[r"^([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s([\w]*\s?)\(?([\w\s]*)\)?$",
                         r"^([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)$",
                        r"^([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(0x[\da-f]*)$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s([\w]*\s?)\(?([\w\s]*)\)?$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(0x[\da-f]*)$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(0x[\da-f]*)$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2}\.[\d]{6})\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s([\w]*\s?)\(?([\w\s]*)\)?$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2}\.[\d]{6})\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s([\w]*\s?)\(?([\w\s]*)\)?$",
                        r"^([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)$",
                        r"^([\d]+)\s*\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)$",
                        r"^([\d]+)\s*\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s([\w]*\s?)\(?([\w\s]*)\)?$",
                        r"^([\d]+)\s*\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(0x[\da-f]*)$",
                        r"^([\d]+)\s*([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s<(0\.\d+)>$",
                        r"^([\d]+)\s*([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s([\w]*\s?)\(?([\w\s]*)\)?\s<(0\.\d+)>$",
                        r"^([\d]+)\s*([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(0x[\da-f]*)\s<(0\.\d+)>$",
                        r"^([\d]+)\s*([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)$",
                        r"^([\d]+)\s*([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(-?[\d]*)\s([\w]*\s?)\(?([\w\s]*)\)?",
                        r"^([\d]+)\s*([\d]{2}:{1}[\d]{2}:{1}[\d]{2})\s\[([\da-f]*)\]\s([_\w\d]+)\(([\w\d\s\"\=\[\]\,\/\.\-\|\}\{\*\<\>\`\\]*)\)\s+=\s+(0x[\da-f]*)"
                        ]
    def __init__(self,sys_call,arguments,return_value,process_id=None,any_explain=None,return_macro=None,exec_time=0,sys_clk_time=0,ip=None,reg_match=None):
        self.sys_call=sys_call
        self.arguments=arguments
        self.return_value=return_value
        self.any_explain=any_explain
        self.exec_time=exec_time
        self.return_macro=return_macro
        self.sys_clk_time=sys_clk_time
        self.ip=ip
        self.process_id=process_id
        self.reg_match=reg_match

        
    
    @classmethod
    def createStrace(cls,data):
        straceObj=None
        if re.match(cls.regular_expressions[0],data):
            match=re.match(cls.regular_expressions[0],data).groups()
            straceObj=cls(sys_call=match[0],arguments=match[1],return_value=match[2],return_macro=match[3],any_explain=match[4],reg_match=cls.regular_expressions[0])
        elif re.match(cls.regular_expressions[1],data):
            match=re.match(cls.regular_expressions[1],data).groups()
            straceObj=cls(sys_call=match[0],arguments=match[1],return_value=match[2],reg_match=cls.regular_expressions[1])
        elif re.match(cls.regular_expressions[2],data):
            match=re.match(cls.regular_expressions[2],data).groups()
            straceObj=cls(sys_call=match[0],arguments=match[1],return_value=match[2],reg_match=cls.regular_expressions[2])
        elif re.match(cls.regular_expressions[3],data):
            match=re.match(cls.regular_expressions[3],data)(match).groups()
            straceObj=cls(sys_clk_time=match[0],sys_call=match[1],arguments=match[2],return_value=match[3],reg_match=cls.regular_expressions[3])
        elif re.match(cls.regular_expressions[4],data):
            match=re.match(cls.regular_expressions[4],data).groups()
            straceObj=cls(sys_clk_time=match[0],sys_call=match[1],arguments=match[2],return_value=match[3],return_macro=match[4],any_explain=match[5],reg_match=cls.regular_expressions[4])
        elif re.match(cls.regular_expressions[5],data):
            match=re.match(cls.regular_expressions[5],data).groups()
            straceObj=cls(sys_clk_time=[0],sys_call=match[1],arguments=match[2],return_value=match[3],reg_match=cls.regular_expressions[5])    
        elif re.match(cls.regular_expressions[6],data):
            match=re.match(cls.regular_expressions[6],data).groups()
            straceObj=cls(sys_clk_time=[0],ip=match[1],sys_call=match[2],arguments=match[3],return_value=match[4],reg_match=cls.regular_expressions[6])    
        elif re.match(cls.regular_expressions[7],data):
            match=re.match(cls.regular_expressions[7],data).groups()
            straceObj=cls(sys_clk_time=match[0],sys_call=match[1],arguments=match[2],return_value=match[3],return_macro=match[4],any_explain=match[5],reg_match=cls.regular_expressions[7])
        elif re.match(cls.regular_expressions[8],data):
            match=re.match(cls.regular_expressions[8],data).groups()
            straceObj=cls(sys_clk_time=match[0],sys_call=match[1],arguments=match[2],return_value=match[3],reg_match=cls.regular_expressions[8])
        elif re.match(cls.regular_expressions[9],data):
            match=re.match(cls.regular_expressions[9],data).groups()
            straceObj=cls(sys_clk_time=match[0],ip=match[1],sys_call=match[2],arguments=match[3],return_value=match[4],reg_match=cls.regular_expressions[9])    
        elif re.match(cls.regular_expressions[10],data):
            match=re.match(cls.regular_expressions[10],data).groups()
            straceObj=cls(sys_clk_time=match[0],ip=match[1],sys_call=match[2],arguments=match[3],return_value=match[4],reg_match=cls.regular_expressions[10])
        elif re.match(cls.regular_expressions[11],data):
            match=re.match(cls.regular_expressions[11],data).groups()
            straceObj=cls(process_id=match[0],sys_call=match[2],ip=match[1],arguments=match[3],return_value=match[4],reg_match=cls.regular_expressions[11])
        elif re.match(cls.regular_expressions[12],data):
            match=re.match(cls.regular_expressions[12],data).groups()
            straceObj=cls(process_id=match[0],sys_call=match[2],ip=match[1],arguments=match[3],return_value=match[4],return_macro=match[5],any_explain=match[6],reg_match=cls.regular_expressions[12])
        elif re.match(cls.regular_expressions[13],data):
            match=re.match(cls.regular_expressions[13],data).groups()
            straceObj=cls(process_id=match[0],sys_call=match[2],ip=match[1],arguments=match[3],return_value=match[4],reg_match=cls.regular_expressions[13])
        elif re.match(cls.regular_expressions[14],data):
            match=re.match(cls.regular_expressions[14],data).groups()
            straceObj=cls(process_id=match[0],sys_clk_time=match[1],sys_call=match[3],ip=match[2],arguments=match[4],return_value=match[5],exec_time=match[6],reg_match=cls.regular_expressions[14])
        elif re.match(cls.regular_expressions[15],data):
            match=re.match(cls.regular_expressions[15],data).groups()
            straceObj=cls(process_id=match[0],sys_clk_time=match[1],sys_call=match[3],ip=match[2],arguments=match[4],return_value=match[5],return_macro=match[6],any_explain=match[7],exec_time=match[8],reg_match=cls.regular_expressions[15])
        elif re.match(cls.regular_expressions[16],data):
            match=re.match(cls.regular_expressions[16],data).groups()
            straceObj=cls(process_id=match[0],sys_clk_time=match[1],sys_call=match[3],ip=match[2],arguments=match[4],return_value=match[5],exec_time=match[6],reg_match=cls.regular_expressions[16])
        elif re.match(cls.regular_expressions[17],data):
            match=re.match(cls.regular_expressions[17],data).groups()
            straceObj=cls(process_id=match[0],sys_clk_time=match[1],sys_call=match[3],ip=match[2],arguments=match[4],return_value=match[5],reg_match=cls.regular_expressions[17])
        elif re.match(cls.regular_expressions[18],data):
            match=re.match(cls.regular_expressions[18],data).groups()
            straceObj=cls(process_id=match[0],sys_clk_time=match[1],sys_call=match[3],ip=match[2],arguments=match[4],return_value=match[5],return_macro=match[6],any_explain=match[7],reg_match=cls.regular_expressions[18])
        elif re.match(cls.regular_expressions[19],data):
            match=re.match(cls.regular_expressions[19],data).groups()
            straceObj=cls(process_id=match[0],sys_clk_time=match[1],sys_call=match[3],ip=match[2],arguments=match[4],return_value=match[5],reg_match=cls.regular_expressions[19])

        if straceObj:
            straceObj.arguments=[arg.strip('"[] {}[]') for arg in straceObj.arguments.split(",") if arg]

        return straceObj
            
    @classmethod
    def fromFile(cls,path):
        path=open(path,'r')
        listOfData=[]
        lines=path.read().split("\n")
        print("Total System Calls Executed Length: {}".format(len(lines)))
        for line in lines:
            obj=StraceOutput.createStrace(line)
            if obj is not None:
                listOfData.append(obj)
        print("Total Line Parsed: {}".format(len(listOfData)))
        return listOfData
    
    def __str__(self):
        args_string=", ".join(self.arguments)
        if self.reg_match==StraceOutput.regular_expressions[0]:
            return "{}({}) = {} {} ({})".format(self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[1]:
            return "{}({}) = {}".format(self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[2]:
            return "{}({}) = {}".format(self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[3]:
            return "{} {}({}) = {}".format(self.sys_clk_time,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[4]:
            return "{} {}({}) = {} {} ({})".format(self.sys_clk_time,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[5]:
            return "{} {}({}) = {}".format(self.sys_clk_time,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[6]:
            return "{} {} {}({}) = {}".format(self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[7]:
            return "{} {}({}) = {} {} ({})".format(self.sys_clk_time,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[8]:
            return "{} {}({}) = {}".format(self.sys_clk_time,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[9]:
            return "{} [{}] {}({}) = {}".format(self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[10]:
            return "{} [{}] {}({}) = {}".format(self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[11]:
            return "{} [{}] {}({}) = {}".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[12]:
            return "{} [{}] {}({}) = {} {} ({})".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[13]:
            return "{} [{}] {}({}) = {}".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[14]:
            return "{} {} [{}] {}({}) = {} <{}>".format(self.process_id,self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value,self.exec_time)
        elif self.reg_match==StraceOutput.regular_expressions[15]:
            return "{} {} [{}] {}({}) = {} {} ({}) <{}>".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain,self.exec_time)
        elif self.reg_match==StraceOutput.regular_expressions[16]:
            return "{} {} [{}] {}({}) = {} <{}>".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value,self.exec_time)
        elif self.reg_match==StraceOutput.regular_expressions[17]:
            return "{} {} [{}] {}({}) = {} ".format(self.process_id,self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[18]:
            return "{} {} [{}] {}({}) = {} {} ({})".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[19]:
            return "{} {} [{}] {}({}) = {}".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)

        
    def __repr__(self):
        args_string=", ".join(self.arguments)
        if self.reg_match==StraceOutput.regular_expressions[0]:
            return "{}({}) = {} {} ({})".format(self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[1]:
            return "{}({}) = {}".format(self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[2]:
            return "{}({}) = {}".format(self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[3]:
            return "{} {}({}) = {}".format(self.sys_clk_time,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[4]:
            return "{} {}({}) = {} {} ({})".format(self.sys_clk_time,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[5]:
            return "{} {}({}) = {}".format(self.sys_clk_time,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[6]:
            return "{} {} {}({}) = {}".format(self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[7]:
            return "{} {}({}) = {} {} ({})".format(self.sys_clk_time,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[8]:
            return "{} {}({}) = {}".format(self.sys_clk_time,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[9]:
            return "{} {} {}({}) = {}".format(self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[10]:
            return "{} {} {}({}) = {}".format(self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[11]:
            return "{} [{}] {}({}) = {}".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[12]:
            return "{} [{}] {}({}) = {} {} ({})".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[13]:
            return "{} [{}] {}({}) = {} {}".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[14]:
            return "{} {} [{}] {}({}) = {} <{}>".format(self.process_id,self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value,self.exec_time)
        elif self.reg_match==StraceOutput.regular_expressions[15]:
            return "{} {} [{}] {}({}) = {} {} ({}) <{}>".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain,self.exec_time)
        elif self.reg_match==StraceOutput.regular_expressions[16]:
            return "{} [{}] {}({}) = {} <{}>".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value,self.exec_time)
        elif self.reg_match==StraceOutput.regular_expressions[17]:
            return "{} {} [{}] {}({}) = {}".format(self.process_id,self.sys_clk_time,self.ip,self.sys_call,args_string,self.return_value)
        elif self.reg_match==StraceOutput.regular_expressions[18]:
            return "{} {} [{}] {}({}) = {} {} ({})".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value,self.return_macro,self.any_explain)
        elif self.reg_match==StraceOutput.regular_expressions[19]:
            return "{} {} [{}] {}({}) = {}".format(self.process_id,self.ip,self.sys_call,args_string,self.return_value)

    
    @classmethod
    def saveJson(cls,data,filename="data.json"):
        with open(filename,'w') as f:
            json.dump([item.__dict__ for item in data],f)

In [2]:
len(StraceOutput.regular_expressions)

20

In [3]:
import pandas as pd

In [4]:
pd.set_option('display.max_rows', None)

In [5]:
normal=pd.read_json('normal.json')

In [6]:
normal=normal.drop(['reg_match','any_explain','exec_time','return_macro','sys_clk_time','reg_match'],axis=1)

In [7]:
normal.to_json('normal_record.json',orient='records')

In [8]:
dangerous_sys_call="""open chown mount fchown setuid setregid setfsgid create_module link lchown symlink chown setgid setgroups setresuid unlink rename fchmod execve setreuid setfsuid setresgid"""

In [9]:
dangerous_sys_call=dangerous_sys_call.split()

In [10]:
new_dangerous_call=[{'sys_call':'open','number':2},
                    {'sys_call':'chown','number':92},
                    {'sys_call':'mount','number':165},
                    {'sys_call':'fchown','number':93},
                    {'sys_call':'setuid','number':105},
                    {'sys_call':'setregid','number':114},
                    {'sys_call':'setfsgid','number':123},
                    {'sys_call':'create_module','number':174},
                    {'sys_call':'link','number':86},
                    {'sys_call':'lchown','number':94},
                    {'sys_call':'symlink','number':88},
                    {'sys_call':'unlink','number':87},
                    {'sys_call':'rename','number':82},
                    {'sys_call':'setgroups',"number":116},
                    {'sys_call':'execve',"number":59},
                    {'sys_call':'chmod',"number":90},
                    {'sys_call':'fchmod','number':91},
                    {'sys_call':'setreuid','number':113},
                    {'sys_call':'setfsuid','number':122},
                    {'sys_call':'bind','number':49},
                    {'sys_call':'connect','number':42},
                    {'sys_call':'recvfrom','number':45},
                    {'sys_call':'recvmsg','number':47},
                    {'sys_call':'recvmmsg','number':299},
                    {'sys_call':'sendto','number':44},
                    {'sys_call':'sendmsg','number':46},
                    {'sys_call':'sendfile','number':40},
                    {'sys_call':'chdir','number':80},
                    {'sys_call':'dup','number':32},
                    {'sys_call':'dup2','number':33}
                    
                   ]

In [11]:
import os


In [12]:
cwd=os.path.abspath('.')
strace_dir=[f for f in os.listdir('./RawData/') if os.path.isdir(os.path.join(cwd,'RawData'))]
if not os.path.isdir(os.path.join(cwd,'pruneData')):
    os.mkdir(os.path.join(cwd,'pruneData'))
for cdir in strace_dir:
    if not os.path.isdir(os.path.join(cwd,'pruneData',cdir)):
        os.mkdir(os.path.join(cwd,'pruneData',cdir))
    strace_ftp_files=[fi for fi in os.listdir(os.path.join(cwd,'RawData',cdir)) if fi.endswith('strace')]
    for file in strace_ftp_files:
        absPath=os.path.join(cwd,'RawData',cdir,file)
        normal=StraceOutput.fromFile(absPath)
        print(absPath)
        ftpFiles=StraceOutput.fromFile(os.path.join(cwd,'RawData',cdir,file))
        StraceOutput.saveJson(ftpFiles,os.path.join(cwd,'pruneData',cdir,file.split('.')[0]+'.json'))


Total System Calls Executed Length: 3023
Total Line Parsed: 2956
/home/bug/PycharmProjects/strace-json-parser/RawData/FTP_Attack/attack1.strace
Total System Calls Executed Length: 3023
Total Line Parsed: 2956
Total System Calls Executed Length: 32913
Total Line Parsed: 32663
/home/bug/PycharmProjects/strace-json-parser/RawData/FTP_Attack/normal2.strace
Total System Calls Executed Length: 32913
Total Line Parsed: 32663
Total System Calls Executed Length: 2018
Total Line Parsed: 1962
/home/bug/PycharmProjects/strace-json-parser/RawData/FTP_Attack/normal10.strace
Total System Calls Executed Length: 2018
Total Line Parsed: 1962
Total System Calls Executed Length: 2250
Total Line Parsed: 2186
/home/bug/PycharmProjects/strace-json-parser/RawData/FTP_Attack/normal1.strace
Total System Calls Executed Length: 2250
Total Line Parsed: 2186
Total System Calls Executed Length: 2018
Total Line Parsed: 1962
/home/bug/PycharmProjects/strace-json-parser/RawData/FTP_Attack/normal8.strace
Total System Ca